<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Python a Ingles: PLBART Baseline**

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio
!pip install -q evaluate
!pip install -q transformers==4.47.0
!pip install -q accelerate==0.27.0
!pip install -q peft
!pip install -q bitsandbytes==0.45.0
!pip install -q sacrebleu
!pip install -q unbabel-comet
!pip install -q rouge_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

dataset = load_dataset("code_x_glue_ct_code_to_text", "python")

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})

In [ ]:
dataset['train'].features

{'id': Value(dtype='int32', id=None),
 'repo': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'original_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'code': Value(dtype='string', id=None),
 'code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'docstring': Value(dtype='string', id=None),
 'docstring_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'sha': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None)}

In [ ]:
print(len(dataset['train']))
print(len(dataset['validation']))
print(len(dataset['test']))

251820
13914
14918


In [ ]:
seed = 42

train_dataset = dataset['train'].shuffle(seed=seed).select(range(200))
validation_dataset = dataset['validation'].shuffle(seed=seed).select(range(1000))
test_dataset = dataset['test'].shuffle(seed=seed).select(range(1000))

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(validation_dataset))
print("Test dataset size:", len(test_dataset))

Training dataset size: 200
Validation dataset size: 1000
Test dataset size: 1000


In [ ]:
train_dataset[0]['code']

'def metapolicy(request, permitted, domains=None):\n    """\n    Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommen

In [ ]:
train_dataset[0]["docstring"]

'Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommended that you not use wildcard\n        domain values.'

In [ ]:
max_tok_length = 512

from transformers import AutoTokenizer

checkpoint = "uclanlp/plbart-large"

tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    padding=True,
    pad_to_multiple_of=8,
    truncation=True,
    max_length=max_tok_length,
    )

In [ ]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["code"],
        text_target = sample["docstring"],
        )
    return model_inputs

In [ ]:
test_dataset_filtered = test_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")

In [ ]:
print("Test dataset size:", len(test_dataset_filtered))

Test dataset size: 930


In [ ]:
tokenized_test_dataset = test_dataset_filtered.map(preprocess_function, batched=True)

In [ ]:
dic = {}
for sample in tokenized_test_dataset:
    sample_length = len(sample['input_ids'])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1,max_tok_length+1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

35   2
36   1
37   3
38   2
39   1
40   2
41   2
42   2
43   3
44   6
45   2
46   5
47   1
48  10
49   4
50   7
51  13
52   3
53   4
54   4
55   4
56   5
57   2
58   5
60   6
61   3
62   7
63   6
64   5
65   8
66   8
67   6
68   6
69   9
70   6
71   5
72   4
73  10
74   5
75   7
76   7
77   4
78   4
79   6
80   7
81   7
82   6
83   2
84   4
85   9
86   6
87   3
88   5
89   8
90   5
91   5
92   8
93   5
94   2
95   3
96   1
97   8
98   3
99   4
100   4
101   4
102   3
103   6
104   6
105   6
106   1
107   7
108   8
109   4
110   6
111   2
112   7
113   2
114   3
115   3
116   8
117   7
118   6
119   1
120   4
121   3
122   9
123   4
124   9
125   3
126   3
127   1
128   4
129   6
130   1
131   1
132   4
133   4
134   6
135   5
136   3
137   4
138   1
139   5
140   7
141   1
142   2
143   6
144   3
145   2
146   2
147   3
148   1
149   2
150   2
151   1
152   5
153   3
154   3
155   2
156   2
157   4
158   4
159   4
160   4
161   2
162   1
163   6
164   3
165   2
166   4
167   3
168   1


In [ ]:
for sample in tokenized_test_dataset.select(range(3)):
    print(sample['input_ids'])
    print(sample['attention_mask'])
    print(sample['labels'])

[134, 1001, 33456, 6329, 33460, 1409, 33463, 1314, 33456, 6329, 33463, 2416, 33456, 6329, 33470, 3861, 988, 2133, 4319, 197, 1836, 3202, 5921, 16471, 2990, 24, 1314, 33456, 6329, 33455, 3484, 292, 105, 2416, 33456, 6329, 96, 188, 307, 33475, 2990, 33455, 2683, 33460, 8385, 33456, 6329, 33459, 111, 2990, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[4489, 197, 1836, 3202, 33455, 2]
[134, 529, 33456, 2939, 33460, 1409, 33463, 18, 33475, 524, 33459, 2013, 3141, 33475, 2133, 27467, 57, 18, 2996, 1203, 33463, 4402, 759, 638, 135, 111, 489, 345, 111, 611, 33455, 872, 374, 9318, 2939, 135, 141, 33473, 33442, 57, 845, 268, 4255, 55, 245, 28292, 33455, 2133, 105, 61, 33455, 474, 33456, 23649, 4071, 111, 611, 61, 4492, 3068, 33455, 4380, 33456, 2292, 292, 105, 18, 258, 61, 4492, 3068, 33455, 125, 4396, 33456, 2939, 33475, 61, 4492, 3068, 33455, 4466, 1756, 292, 111, 61, 4492, 3068, 334

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config
    )


`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

### **Evaluación**

In [ ]:
test_batch_size = 32
batch_tokenized_test = tokenized_test_dataset.batch(test_batch_size)

Batching examples:   0%|          | 0/930 [00:00<?, ? examples/s]

In [ ]:
number_of_batches = len(batch_tokenized_test["code"])
output_sequences = []
sources = []

for i in range(number_of_batches):

    inputs = tokenizer(
        batch_tokenized_test["code"][i],
        max_length=max_tok_length,
        truncation=True,
        return_tensors="pt",
        padding=True,
    )
    output_batch = model.generate(
        input_ids=inputs["input_ids"].cuda(),
        attention_mask=inputs["attention_mask"].cuda(),
        max_length=max_tok_length,
        num_beams=1,
        do_sample=False
        )
    output_sequences.extend(output_batch.cpu())

In [ ]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [ ]:
references = tokenizer.batch_decode(tokenized_test_dataset["labels"], skip_special_tokens=True)

In [ ]:
sources = test_dataset_filtered['code']

In [ ]:
decoded_preds[:1]

['build_attrs( self, base_attrs, extra_=None): """Build an attribute dictionary.""" attrs = base_attrs.copy() if extra_attrs is not None: attrs.update(extra_attrs) return attrs = self.build_attrs(self,,),']

In [ ]:
references[:1]

['Build an attribute dictionary.']

In [ ]:
sources[:1]

['def build_attrs(self, base_attrs, extra_attrs=None):\n        """Build an attribute dictionary."""\n        attrs = base_attrs.copy()\n        if extra_attrs is not None:\n            attrs.update(extra_attrs)\n        return attrs']

In [ ]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]
sources_clean = [normalizer(text) for text in sources]

In [ ]:
from evaluate import load

metric = load("sacrebleu")

In [ ]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 20.3


In [ ]:
from evaluate import load

rouge_metric = load("rouge")
comet_metric = load("comet")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
comet_score = comet_metric.compute(predictions = decoded_preds_clean, references = references_clean, sources = sources_clean)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 51.69 %


In [ ]:
rouge_score = rouge_metric.compute(predictions = decoded_preds_clean, references = references_clean)

In [ ]:
print(f"Rouge 1: {rouge_score['rouge1'] * 100:.2f} %")
print(f"Rouge 2: {rouge_score['rouge2'] * 100:.2f} %")
print(f"Rouge L: {rouge_score['rougeL'] * 100:.2f} %")

Rouge 1: 35.91 %
Rouge 2: 30.68 %
Rouge L: 34.91 %
